This notebook is mainly to test out teamwork api, its nothing permanent

In [4]:
import teamwork_client
import json
import datetime
from dotenv import load_dotenv
import os

In [36]:
def print_response(response):
    if 200 <= response.status_code < 300:
        try:
            formatted_json = json.dumps(response.json(), indent=4)
            print(formatted_json)
        except ValueError:
            print("Response content is not in JSON format.")
    else:
        print(f"Failed to retrieve data: {response.status_code} - {response.text}")

In [14]:
def format_datetime(time_str):
    today = datetime.datetime.now().date()
    # Parse the time
    time = datetime.datetime.strptime(time_str, '%H:%M:%S').time()
    # Combine date and time
    datetime_obj = datetime.datetime.combine(today, time)
    # Format as ISO 8601 with 'Z' (indicative of UTC)
    return datetime_obj.strftime('%Y-%m-%dT%H:%M:%S')

In [3]:
load_dotenv()

True

In [5]:
apiToken = os.getenv('TEAMWORK_API')
baseUrl = os.getenv('TEAMWORK_BASEURL')

In [4]:
PROJECT_NAME = 'Resonant Solutions'
TASK = "Labourly"
SUBTASK = "Coding"

In [5]:
client = teamwork_client.TeamworkClient(baseUrl, apiToken)

In [6]:
query_param ={
    'searchTerm' : PROJECT_NAME
}
response_project = client.get_projects(query_param)

In [7]:
if(response_project.status_code != 200):
    raise SystemExit("Exiting due to unsuccessful response. Failed to retrieve data: {response_project.status_code} - {response_project.text}")


project_data = response_project.json().get('projects', [])
projectId = None

for project in project_data:
    if(project['name'] == PROJECT_NAME):
        projectId = project['id']

if(projectId):
    print(f"Project ID for '{PROJECT_NAME}': {projectId}")
else:
    print(f"Project with name '{PROJECT_NAME}' not found.")

Project ID for 'Resonant Solutions': 865864


In [8]:
response_task_list = client.get_tasks_list(projectId=projectId)


In [9]:
if(response_task_list.status_code != 200):
    raise SystemExit("Exiting due to unsuccessful response. Failed to retrieve data: {response_task_list.status_code} - {response_task_list.text}")


task_list_data = response_task_list.json().get('tasklists', [])
taskListId = None

for task in task_list_data:
    if(task['name'] == TASK):
        taskListId = task['id']
        break

if(taskListId):
    print(f"Task List ID for '{TASK}': {taskListId}")
else:
    print(f"Task List with name '{TASK}' not found.")

Task List ID for 'Labourly': 2877254


In [10]:
response_task = client.get_tasks_from_tasks_list(taskListId,None)


In [11]:
if(response_task.status_code != 200):
    raise SystemExit("Exiting due to unsuccessful response. Failed to retrieve data: {response_task.status_code} - {response_task.text}")


tasks_data = response_task.json().get('tasks', [])
taskId = None

for task in tasks_data:
    if(task['name'] == SUBTASK):
        taskId = task['id']
        break

if(taskId):
    print(f"Task List ID for '{SUBTASK}': {taskId}")
else:
    print(f"Task List with name '{SUBTASK}' not found.")

Task List ID for 'Coding': 33098642


In [33]:
timePayload = {
    "timelog": {
        "description": "This is a test",
        "hours": 7,
        "isBillable": False,
        "isUtc": True,
        "minutes": 0,
        "projectId": projectId,
        "taskId": taskId,
        "date": datetime.datetime.now().strftime('%Y-%m-%d'),
        "time": "14:00:00"
    }
}


In [ ]:
respone_time_entries = client.post_time_entry_from_task(taskId, timePayload)
print_response(respone_time_entries)